#Instalamos pytorch


In [1]:
#pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

#Clonamos el repositorio para obtener el dataset

In [2]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 10153 (delta 25), reused 39 (delta 16), pack-reused 10096
Receiving objects: 100% (10153/10153), 236.95 MiB | 36.84 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Checking out files: 100% (10108/10108), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

# Importar las librerías

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importar el dataset


In [4]:
movies = pd.read_csv("/content/deeplearning-az/datasets/Part 6 - AutoEncoders (AE)/ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("/content/deeplearning-az/datasets/Part 6 - AutoEncoders (AE)/ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("/content/deeplearning-az/datasets/Part 6 - AutoEncoders (AE)/ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')


# Preparar el conjunto de entrenamiento y elconjunto de testing

In [5]:
training_set = pd.read_csv("/content/deeplearning-az/datasets/Part 6 - AutoEncoders (AE)/ml-100k/u1.base", sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("/content/deeplearning-az/datasets/Part 6 - AutoEncoders (AE)/ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

# Obtener el número de usuarios y de películas

In [6]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna


In [7]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
training_set = convert(training_set)
test_set = convert(test_set)

# Convertir los datos a tensores de Torch

In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Crear la arquitectura de la Red Neuronal

In [10]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [11]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Entrenar el SAE

In [12]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
            mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
            s += 1.
            optimizer.step()
    print("Epoch: "+str(epoch)+", Loss: "+str(train_loss/s))


Epoch: 1, Loss: tensor(1.7711)
Epoch: 2, Loss: tensor(1.0967)
Epoch: 3, Loss: tensor(1.0534)
Epoch: 4, Loss: tensor(1.0385)
Epoch: 5, Loss: tensor(1.0307)
Epoch: 6, Loss: tensor(1.0267)
Epoch: 7, Loss: tensor(1.0237)
Epoch: 8, Loss: tensor(1.0218)
Epoch: 9, Loss: tensor(1.0209)
Epoch: 10, Loss: tensor(1.0196)
Epoch: 11, Loss: tensor(1.0189)
Epoch: 12, Loss: tensor(1.0183)
Epoch: 13, Loss: tensor(1.0179)
Epoch: 14, Loss: tensor(1.0172)
Epoch: 15, Loss: tensor(1.0174)
Epoch: 16, Loss: tensor(1.0169)
Epoch: 17, Loss: tensor(1.0168)
Epoch: 18, Loss: tensor(1.0164)
Epoch: 19, Loss: tensor(1.0165)
Epoch: 20, Loss: tensor(1.0162)
Epoch: 21, Loss: tensor(1.0162)
Epoch: 22, Loss: tensor(1.0159)
Epoch: 23, Loss: tensor(1.0160)
Epoch: 24, Loss: tensor(1.0160)
Epoch: 25, Loss: tensor(1.0159)
Epoch: 26, Loss: tensor(1.0156)
Epoch: 27, Loss: tensor(1.0154)
Epoch: 28, Loss: tensor(1.0150)
Epoch: 29, Loss: tensor(1.0126)
Epoch: 30, Loss: tensor(1.0110)
Epoch: 31, Loss: tensor(1.0095)
Epoch: 32, Loss: 

# Evaluar el conjunto de test en nuestro SAE

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae.forward(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
        mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
        test_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
        s += 1.

In [14]:
print("Test Loss: "+str(test_loss/s))

Test Loss: tensor(0.9549)
